# 02: Digital Modulation

This notebook explores common digital modulation schemes used in SDR.

## Learning Objectives
- Understand PSK, QAM, and FSK modulation
- Compare constellation diagrams
- Analyze spectral efficiency vs robustness tradeoffs
- Use R4W to generate modulated signals

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from r4w_python import R4W, plot_constellation, plot_spectrum, compute_spectrum

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

## Phase Shift Keying (PSK)

PSK encodes data by varying the phase of the carrier:

| Modulation | Phases | Bits/Symbol | 
|------------|--------|-------------|
| BPSK | 2 (0°, 180°) | 1 |
| QPSK | 4 (45°, 135°, 225°, 315°) | 2 |
| 8PSK | 8 (evenly spaced) | 3 |

In [ ]:
# Generate ideal PSK constellation points
def psk_constellation(m):
    """Generate M-PSK constellation points."""
    return np.exp(1j * 2 * np.pi * np.arange(m) / m)

bpsk = psk_constellation(2)
qpsk = psk_constellation(4) * np.exp(1j * np.pi/4)  # 45 degree offset
psk8 = psk_constellation(8)

In [ ]:
# Plot PSK constellations
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for ax, (name, const) in zip(axes, [('BPSK', bpsk), ('QPSK', qpsk), ('8PSK', psk8)]):
    plot_constellation(const, title=f"{name} Constellation", ax=ax, marker_size=100)
    
    # Add bit labels
    bits_per_sym = int(np.log2(len(const)))
    for i, c in enumerate(const):
        label = format(i, f'0{bits_per_sym}b')
        ax.annotate(label, (c.real + 0.1, c.imag + 0.1))

plt.tight_layout()
plt.show()

## Quadrature Amplitude Modulation (QAM)

QAM encodes data using both amplitude AND phase:

| Modulation | Points | Bits/Symbol |
|------------|--------|-------------|
| 16-QAM | 16 | 4 |
| 64-QAM | 64 | 6 |
| 256-QAM | 256 | 8 |

In [ ]:
def qam_constellation(m):
    """Generate M-QAM constellation (square)."""
    k = int(np.sqrt(m))
    points = []
    for i in range(k):
        for j in range(k):
            re = 2*i - (k-1)
            im = 2*j - (k-1)
            points.append(complex(re, im))
    return np.array(points) / np.sqrt(2*(k-1)**2/3)  # Normalize

qam16 = qam_constellation(16)
qam64 = qam_constellation(64)

In [ ]:
# Plot QAM constellations
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

plot_constellation(qam16, title="16-QAM Constellation", ax=axes[0], marker_size=50)
plot_constellation(qam64, title="64-QAM Constellation", ax=axes[1], marker_size=20)

plt.tight_layout()
plt.show()

## Noise Impact on Constellations

Higher-order modulations pack more bits but are more susceptible to noise.

In [ ]:
# Add noise to constellations
def add_awgn(symbols, snr_db):
    """Add AWGN noise at given SNR."""
    signal_power = np.mean(np.abs(symbols)**2)
    noise_power = signal_power / (10**(snr_db/10))
    noise = np.sqrt(noise_power/2) * (np.random.randn(len(symbols)) + 
                                       1j * np.random.randn(len(symbols)))
    return symbols + noise

# Generate random symbols
np.random.seed(42)
qpsk_tx = qpsk[np.random.randint(0, 4, 200)]
qam16_tx = qam16[np.random.randint(0, 16, 200)]

snr = 15  # dB

In [ ]:
# Compare QPSK and 16-QAM with noise
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

qpsk_rx = add_awgn(qpsk_tx, snr)
qam16_rx = add_awgn(qam16_tx, snr)

plot_constellation(qpsk_rx, title=f"QPSK @ {snr} dB SNR", ax=axes[0])
plot_constellation(qam16_rx, title=f"16-QAM @ {snr} dB SNR", ax=axes[1])

plt.tight_layout()
plt.show()

print("Notice how 16-QAM points start to overlap while QPSK remains distinguishable.")

## Frequency Shift Keying (FSK)

FSK encodes data by switching between discrete frequencies.

Unlike PSK/QAM, FSK is constant envelope (good for nonlinear amplifiers).

In [ ]:
# Generate 2-FSK signal
sample_rate = 48000
symbol_rate = 1000
samples_per_symbol = sample_rate // symbol_rate

# Frequencies for 0 and 1
f0 = 1000  # Hz for bit 0
f1 = 2000  # Hz for bit 1

bits = np.array([0, 1, 1, 0, 1, 0, 0, 1])
fsk_signal = []

for bit in bits:
    freq = f1 if bit else f0
    t = np.arange(samples_per_symbol) / sample_rate
    fsk_signal.extend(np.exp(2j * np.pi * freq * t))

fsk_signal = np.array(fsk_signal)

In [ ]:
# Plot FSK
fig, axes = plt.subplots(2, 1, figsize=(12, 6))

# Time domain
t_ms = np.arange(len(fsk_signal)) / sample_rate * 1000
axes[0].plot(t_ms, fsk_signal.real)
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Amplitude')
axes[0].set_title('2-FSK Time Domain')

# Spectrum
freqs, power = compute_spectrum(fsk_signal, fft_size=1024)
plot_spectrum(power, freqs * sample_rate, sample_rate, title='2-FSK Spectrum', ax=axes[1])

plt.tight_layout()
plt.show()

## Modulation Comparison

| Modulation | Bits/Symbol | Spectral Efficiency | Noise Tolerance | Use Case |
|------------|-------------|---------------------|-----------------|----------|
| BPSK | 1 | Low | High | Deep space, low SNR |
| QPSK | 2 | Medium | Medium-High | Satellites, CDMA |
| 16-QAM | 4 | High | Medium | WiFi, Cable |
| 64-QAM | 6 | Very High | Low | High-rate WiFi |
| 2-FSK | 1 | Low | High | IoT, simple systems |

## Exercises

1. At what SNR does 16-QAM become undecodable? (Hint: when clusters overlap significantly)

2. Generate a 4-FSK signal with 4 frequencies.

3. Calculate the minimum distance between constellation points for QPSK vs 16-QAM.

In [ ]:
# Your code here
